In [ ]:
!pip install hdbscan
!pip install umap-learn
!pip install squarify

# Libraries

In [ ]:
import umap
import random
import hdbscan
import squarify  
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.cm as cm
import plotly.express as px
import category_encoders as ce
import matplotlib.pyplot as plt 


from sklearn.svm import SVC
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy 
from sklearn.decomposition import PCA
from scipy.spatial import distance_matrix 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cluster import OPTICS, cluster_optics_dbscan
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [ ]:
df = pd.read_csv('/kaggle/input/ecommerce-dataset-for-predictive-marketing-2023/ECommerce_consumer behaviour.csv')

In [ ]:
df

In [ ]:
for i in df.columns:
    print(df[i].value_counts())

In [ ]:
df.shape

In [ ]:
df.describe().T
# This mean that we have only 9 numerical column 

In [ ]:
df.describe().T
# This mean that we have only 9 numerical column 

In [ ]:
df.describe().T
# This mean that we have only 9 numerical column 

In [ ]:
df.nunique()

In [ ]:
df.hist(figsize=(15,10))
plt.subplots_adjust(hspace=0.5)

As we can see many of order numbers are range from 0-10 from each product and the max is 100. many of customers order product in 9 Am- 22 PM of day. most of the customers routin order is between 2-10 days.

In [ ]:
# Correlation Between Continous & Target
plt.figure(figsize=(15,10))
sns.heatmap(df.corr(), annot=True)
plt.show()

In [ ]:
df['order_hour_of_day'].value_counts()

plt.figure(figsize = (10,7))
sns.set_style("ticks")
sns.countplot(data=df, x=df['order_hour_of_day'], alpha=0.6)
plt.title("Busiest times of the day", fontsize=14)
plt.xlabel("Hour of the day", fontsize=12)
plt.ylabel("Number of orders", fontsize=12)
plt.show()

In [ ]:
departments = df['department'].value_counts()

departments_df = pd.DataFrame(departments).reset_index()

departments_df.columns = ['Department', 'Order Count']

top_departments = departments_df.sort_values(by='Order Count', ascending=False).head()

plt.figure(figsize = (10,7))
sns.set_style("ticks")
sns.barplot(data=top_departments, x="Order Count", y="Department", palette = 'flare', alpha=0.6)
plt.title("Top 5 most popular departments", fontsize=14)
plt.xlabel("Number of orders", fontsize=12)
plt.ylabel("Department name", fontsize=12)
plt.show()

In [ ]:
products = df['product_name'].value_counts()

products_df = pd.DataFrame(products).reset_index()

products_df.columns = ['Product', 'Order Count']

top_products = products_df.sort_values(by='Order Count', ascending=False).head()
plt.figure(figsize = (10,7))
sns.set_style("ticks")
sns.barplot(data=top_products, x="Order Count", y="Product", palette = 'flare', alpha=0.6)
plt.title("Top 5 most popular products", fontsize=14)
plt.xlabel("Number of orders", fontsize=12)
plt.ylabel("Product name", fontsize=12)
plt.show()


In [ ]:
# Crosstab to check distribution of products in each department
product_dept_df = pd.crosstab(df['department'], df['product_name'])
product_dept_df.head()

In [ ]:
product_dept_df.idxmax(axis=1).to_frame(name="Most popular Item")

In [ ]:
product_reordered_df = df.groupby('product_name')['reordered'].count().reset_index().sort_values(by='reordered', ascending=False)
product_reordered_df


In [ ]:
plt.figure(figsize = (10,7))
sns.set_style("ticks")
sns.barplot(data=product_reordered_df.head(5), x="reordered", y="product_name", palette = 'flare', alpha=0.6)
plt.title("Top 5 most reordered products", fontsize=14)
plt.xlabel("Number of reorders", fontsize=12)
plt.ylabel("Product name", fontsize=12)
plt.show()


In [ ]:
# AGGREGATING & GROUPING VALUES TO VISUALIZE PURCHASING BEHAVIOUR
grouped = df.groupby("order_id")["add_to_cart_order"].aggregate("max").reset_index()
grouped = grouped.add_to_cart_order.value_counts()

sns.set_style('dark')
sns.set_palette("rocket_r")
f, ax = plt.subplots(figsize=(15, 12))
sns.barplot(x=grouped.index, y=grouped.values, ax=ax)
ax.grid(True, axis='y')

plt.xticks(rotation='vertical', fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel('Number of unique orders', fontsize=15)
plt.xlabel('Number of products added to cart', fontsize=15)
plt.title('Purchasing Behavior by Number of Products Added to Cart', fontsize=18)
plt.xlim(0, 35)  # limit the X axis values to 35
plt.show()

In [ ]:
# TIME OF THE DAY WHEN THE ORDER WAS MADE
grouped = df.groupby('order_hour_of_day', as_index=True).agg({'user_id':'count'}).sort_values(by='user_id',ascending=False)


f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation='vertical')
sns.barplot(x = grouped.index, y = grouped.user_id)
sns.color_palette("rocket_r", 10)

plt.ylabel('Number of unique orders', fontsize=13)
plt.xlabel('Time of the day', fontsize=13)

for i in ax.patches:
    ax.text(i.get_x()+0.15, i.get_height()+50, str(round(i.get_height())), fontsize=10, color='black')
plt.show()

plt.show()

In [ ]:
def order_number_group(num_orders):
    ranges = [(1, 10), (11, 20), (21, 30), (31, 40), (41, 50),(51, 60),(61, 70),(71, 80),(81, 90),(91, 100)]
    for r in ranges:
        if num_orders in range(r[0], r[1]+1):
            return f"{r[0]}-{r[1]} orders"
    return "More than 100 orders"

In [ ]:
top_products = df.groupby('product_name')['user_id'].count().sort_values(ascending=False).head(10)
colors = ['#4C72B0', '#55A868', '#C44E52', '#8172B2', '#CCB974', '#64B5CD', '#4E79A7', '#E69F00', '#F0E442', '#59A14F', '#8C8C8C', '#9C755F', '#EDB8A7', '#BDBDBD', '#000000']
ax = top_products.plot(kind='bar', title='Top 10 Products', color=colors, figsize=(15, 15))
plt.xlabel('Product Name',fontsize=13)
plt.ylabel('Number of Orders',fontsize=13)
for i in ax.patches:
    ax.text(i.get_x()+0.15, i.get_height()+50, str(round(i.get_height())), fontsize=10, color='black')
plt.show()

In [ ]:
bottom_products = df.groupby('product_name')['user_id'].count().sort_values(ascending=False).tail(10)
colors = ['#4C72B0', '#55A868', '#C44E52', '#8172B2', '#CCB974', '#64B5CD', '#4E79A7', '#E69F00', '#F0E442', '#59A14F', '#8C8C8C', '#9C755F', '#EDB8A7', '#BDBDBD', '#000000']
ax = bottom_products.plot(kind='bar', title='Bottom 10 Products', color=colors, figsize=(15, 15))
plt.xlabel('Product Name',fontsize=13)
plt.ylabel('Number of Orders',fontsize=13)
for i in ax.patches:
    ax.text(i.get_x()+0.15, i.get_height()+50, str(round(i.get_height())), fontsize=10, color='black')
plt.show()

In [ ]:
# # TREEMAP VISUALIZATION OF ITEMS
# df.dropna(inplace=True)
# fig, ax = plt.subplots(figsize=(15, 15))
# fig = squarify.plot(sizes=df['product_name'].value_counts(), label=df['product_name'])


# Data Cleaning

## Remove Outlier

In [ ]:
def remove_rare_categories(df, categorical_columns, threshold):
    """
    Remove rare categories from categorical columns in a DataFrame.
    
    Args:
        df (pandas.DataFrame): Input DataFrame.
        categorical_columns (list): List of categorical column names to analyze.
        threshold (float): Threshold to determine rare categories (e.g., 0.01 for 1% occurrence).
    
    Returns:
        pandas.DataFrame: DataFrame with rare categories removed.
    """
    df_cleaned = df.copy()
    for column in categorical_columns:
        category_counts = df_cleaned[column].value_counts(normalize=True)
        rare_categories = category_counts[category_counts < threshold].index
        df_cleaned = df_cleaned[~df_cleaned[column].isin(rare_categories)]
    
    return df_cleaned

In [ ]:
columns = ['order_dow','order_dow','product_id','add_to_cart_order','reordered','department_id','department','product_name']
threshold = 0.001
df = remove_rare_categories(df,columns,threshold)

## Missing value handling

In [ ]:
def fill_missing_values(df, categorical_columns, method='most_frequent'):
    """
    Fill missing values in categorical columns of a DataFrame.

    Args:
        df (pandas.DataFrame): Input DataFrame.
        categorical_columns (list): List of categorical column names to fill missing values.
        method (str): Method for filling missing values. Options: 'most_frequent', 'unknown'.
            Default is 'most_frequent'.

    Returns:
        pandas.DataFrame: DataFrame with missing values filled in categorical columns.
    """
    df_filled = df.copy()
    for column in categorical_columns:
        if method == 'most_frequent':
            df_filled[column].fillna(df_filled[column].mode().iloc[0], inplace=True)
        elif method == 'unknown':
            df_filled[column].fillna('Unknown', inplace=True)
    
    return df_filled


In [ ]:
df = fill_missing_values(df,df.columns)

# Sampling

In [ ]:
def reservoir_sampling(data, k):
    sample = []

    # Fill the sample array with the first k elements
    for i in range(k):
        sample.append(data.iloc[i])

    # Replace elements in the sample with a decreasing probability
    for i in range(k, len(data)):
        j = random.randint(0, i)  # Generate a random index

        if j < k:
            sample[j] = data.iloc[i]  # Replace element at random index with new element from data

    return pd.DataFrame(sample)


In [ ]:
df_sample = reservoir_sampling(df,100000)

Check the distribution of main data and sampling data

In [ ]:
df_sample.hist(figsize=(15,10))
plt.subplots_adjust(hspace=0.5)

In [ ]:
df.hist(figsize=(15,10))
plt.subplots_adjust(hspace=0.5)

## Hypothesis check

 We want to be more confident aboat the distribution of data_sampler and main data

In [ ]:
import numpy as np
from scipy import stats

def compare_distributions(data1, data2):
    """
    Compare the distributions of two datasets using statistical tests.

    Args:
        data1 (array-like): First dataset.
        data2 (array-like): Second dataset.

    Returns:
        bool: True if the distributions are similar, False otherwise.
    """
    # Perform Kolmogorov-Smirnov test
    ks_statistic, ks_pvalue = stats.ks_2samp(data1, data2)

    # Perform Mann-Whitney U test
    mw_u_statistic, mw_pvalue = stats.mannwhitneyu(data1, data2, alternative='two-sided')

    # Set the significance level
    alpha = 0.05

    # Compare p-values to the significance level
    if ks_pvalue > alpha and mw_pvalue > alpha:
        return True  # Distributions are similar
    else:
        return False  # Distributions are different


In [ ]:
for i in df.columns:
    try:
        print('Is dis of data_sample and data in col {} same :'.format(i),compare_distributions(df[i],df_sample[i]))
    except:
        continue

# Feature Engineering

In [ ]:

encoder = LabelEncoder()

# Encode the columns
df_sample['department'] = encoder.fit_transform(df_sample['department'])
df_sample['product_name'] = encoder.fit_transform(df_sample['product_name'])
# df_sample['order_id'] = encoder.fit_transform(df_sample['order_id'])
# df_sample['user_id'] = encoder.fit_transform(df_sample['user_id'])

# Feature Scaling

In [ ]:
scaler = StandardScaler()
df_sample = scaler.fit_transform(df_sample)


# Models

## Visualiztion of the models

In [ ]:

def tsne_umap_pca(df,labels):
    # TSNE Reduction
    X_embedded = TSNE(n_components=3, learning_rate='auto',
                  init='random', perplexity=3).fit_transform(df)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(X_embedded[:, 0], X_embedded[:, 1],c=labels,cmap='jet')
    plt.show()
    
    # PCA Reduction
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(df)

    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title('PCA Scatter Plot')

    cbar = plt.colorbar()
    cbar.set_label('Cluster')

    plt.show()


## KMEANS

In [ ]:

clusters = []

for i in range(1, 30):
    km = KMeans(n_clusters=i).fit(df_sample)
    clusters.append(km.inertia_)
    


In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
sns.lineplot(x=list(range(1, 30)), y=clusters, ax=ax)
ax.set_title('Searching for Elbow')
ax.set_xlabel('Clusters')
ax.set_ylabel('Inertia')

plt.show()

In [ ]:
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_sample)

range_n_clusters = [4,5,6,10,15,20,25]

for n_clusters in range_n_clusters:
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    ax1.set_xlim([-0.1, 1])

    ax1.set_ylim([0, len(df_sample) + (n_clusters + 1) * 10])

    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(df_sample)

    silhouette_avg = silhouette_score(df_sample, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    sample_silhouette_values = silhouette_samples(df_sample, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(df_pca[:, 0], df_pca[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    centers = clusterer.cluster_centers_

    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

# Hirerachical Clustering

We need sample from df_sample because the algoramative method doesnt work with 100k data

In [ ]:
df_ssample = reservoir_sampling(pd.DataFrame(df_sample),10000)

In [ ]:
df_ssample

In [ ]:
from sklearn.cluster import AgglomerativeClustering 

agglom = AgglomerativeClustering(n_clusters=5, linkage='average').fit(df_ssample)

pca = PCA(n_components=3)
df_pca = pd.DataFrame(pca.fit_transform(df_ssample))

df_pca['Labels'] = agglom.labels_
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df_pca, x=0, y=1, hue="Labels")
plt.title('Agglomerative with 5 Clusters')
plt.show()

plt.figure(figsize=(12, 8))
sns.scatterplot(data=df_pca, x=1, y=2, hue="Labels")
plt.title('Agglomerative with 5 Clusters')
plt.show()

plt.figure(figsize=(12, 8))
sns.scatterplot(data=df_pca, x=0, y=2, hue="Labels")
plt.title('Agglomerative with 5 Clusters')
plt.show()

In [ ]:

dist = distance_matrix(df_ssample, df_ssample)
print(dist)

In [ ]:
Z = hierarchy.linkage(dist, 'complete')


In [ ]:
plt.figure(figsize=(18, 50))
dendro = hierarchy.dendrogram(Z, leaf_rotation=0, leaf_font_size=12, orientation='right')

# DBSCAN

In [ ]:
# df_ssample = reservoir_sampling(df_sample,10000)

In [ ]:
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_ssample)
scaler = StandardScaler()
df_ssample = scaler.fit_transform(df_ssample)


In [ ]:
range_n_clusters = [5,15,30,45]
range_e = [1,2,3,4,5]
for eps in range_e:
    for n_clusters in range_n_clusters:
        try:
            fig, (ax1, ax2) = plt.subplots(1, 2)
            fig.set_size_inches(18, 7)

            ax1.set_xlim([-0.1, 1])
            ax1.set_ylim([0, len(df_ssample) + (n_clusters + 1) * 10])

            clusterer = DBSCAN(eps=eps, min_samples=n_clusters)
            cluster_labels = clusterer.fit_predict(df_ssample)

            silhouette_avg = silhouette_score(df_ssample, cluster_labels)
            print("For n_clusters =", n_clusters,
                  "The average silhouette_score is :", silhouette_avg)

            sample_silhouette_values = silhouette_samples(df_ssample, cluster_labels)

            y_lower = 10
            for i in range(n_clusters):
                ith_cluster_silhouette_values = \
                    sample_silhouette_values[cluster_labels == i]

                ith_cluster_silhouette_values.sort()

                size_cluster_i = ith_cluster_silhouette_values.shape[0]
                y_upper = y_lower + size_cluster_i

                color = cm.nipy_spectral(float(i) / n_clusters)
                ax1.fill_betweenx(np.arange(y_lower, y_upper),
                                  0, ith_cluster_silhouette_values,
                                  facecolor=color, edgecolor=color, alpha=0.7)

                ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

                y_lower = y_upper + 10  # 10 for the 0 samples

            ax1.set_title("The silhouette plot for the various clusters.")
            ax1.set_xlabel("The silhouette coefficient values")
            ax1.set_ylabel("Cluster label")

            ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

            ax1.set_yticks([])  # Clear the yaxis labels / ticks
            ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

            colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
            ax2.scatter(df_ssample[:, 0], df_ssample[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                        c=colors, edgecolor='k')

            ax2.set_title("The visualization of the clustered data.")
            ax2.set_xlabel("Feature space for the 1st feature")
            ax2.set_ylabel("Feature space for the 2nd feature")

            plt.suptitle(("Silhouette analysis for DSCAN clustering on sample data "
                          "with n_clusters = %d" % n_clusters),
                         fontsize=14, fontweight='bold')
        except:
            continue
plt.show()

# HDBSCAN

Why HDBSCAN? HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise) is a powerful density-based clustering algorithm that offers several advantages in comparison to other clustering techniques. Here are some reasons why HDBSCAN is often preferred:

Robust to parameter selection: HDBSCAN automatically determines the number of clusters based on the density and distribution of data points. This eliminates the need for users to specify the number of clusters beforehand, which can be challenging in many cases. HDBSCAN uses a hierarchical approach to identify clusters of different densities and allows for capturing both dense and sparse regions in the data.

Ability to handle varying cluster densities and shapes: HDBSCAN can effectively handle clusters of different densities, including clusters with irregular shapes and varying densities within a single cluster. It utilizes the concept of core samples, which are data points that are densely connected, and it allows for the formation of clusters based on the density connectivity of these core samples.

Noise detection and handling: HDBSCAN can identify and label noise or outliers as "noise" rather than forcing them into clusters. This is particularly valuable in scenarios where the dataset contains noisy or irrelevant data points that do not belong to any meaningful clusters.

Efficiency and scalability: HDBSCAN employs efficient data structures, such as the kd-tree, to speed up the clustering process. It can handle large datasets efficiently and scale well with increasing data size, making it suitable for applications with a large number of data points.

Flexibility in distance metrics: HDBSCAN can work with various distance metrics, enabling it to handle different types of data and problem domains. It allows users to define the proximity measure that is most appropriate for their data, whether it is numerical, categorical, or mixed.

Hierarchical clustering visualization: HDBSCAN produces a hierarchical representation of clusters, known as a dendrogram or cluster tree. This visualization provides insights into the hierarchy of clusters and allows users to explore different levels of cluster granularity.

In [ ]:
%%time

X_embedded = TSNE(n_components=3, learning_rate='auto',
                  init='random', perplexity=3).fit_transform(df_ssample)

In [ ]:
%%time
clusterer_train = hdbscan.HDBSCAN(prediction_data=True).fit(df_ssample)
u_train, counts_train = np.unique(clusterer_train.labels_, return_counts=True)

# clusterer_test = hdbscan.HDBSCAN(prediction_data=True, min_cluster_size = 200 if DEBUG else 1000).fit(embedding_test)
# u_test, counts_test = np.unique(clusterer_test.labels_, return_counts=True)

print(u_train)
print(counts_train)



In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], s=5, c=clusterer_train.labels_, edgecolors='none', cmap='jet');

# OPTICS

Why OPTICS? OPTICS (Ordering Points To Identify the Clustering Structure) is a density-based clustering algorithm that is often used as an alternative to other popular clustering algorithms like k-means or hierarchical clustering. OPTICS offers several advantages and can be beneficial in certain scenarios:

Density-based clustering: OPTICS is a density-based clustering algorithm, which means it can identify clusters of arbitrary shape and does not rely on predefined cluster centroids or assumptions about cluster geometry. It is effective in identifying clusters of varying densities, clusters with irregular shapes, and clusters embedded within larger clusters.

Robust to parameter selection: OPTICS does not require the upfront specification of the number of clusters, unlike k-means or some hierarchical clustering algorithms. This can be advantageous when the optimal number of clusters is unknown or difficult to determine. OPTICS can provide a cluster ordering or reachability plot that allows users to explore different cluster granularity levels.

Cluster reachability analysis: OPTICS produces a reachability plot, which provides insights into the density-based structure of the data. The reachability plot reveals the varying density levels of the data points and can help in understanding the hierarchical relationships between clusters and detecting noise or outliers. This information is not readily available in many other clustering algorithms.

Flexibility in distance metric: OPTICS can work with various distance metrics, allowing users to define the proximity measure that suits their data and problem domain. This flexibility enables the algorithm to handle different types of data, including numerical, categorical, or mixed data.

Handling large datasets: OPTICS can handle large datasets efficiently by utilizing an indexing structure called the "OPTICS ordering." This structure allows for incremental processing and avoids the need to calculate the entire pairwise distance matrix, making it scalable to datasets with a large number of data points.

In [ ]:
df_ssample.shape

In [ ]:
X = df_sample

In [ ]:
%%time
clust = OPTICS(min_samples=50, xi=0.05, min_cluster_size=0.05)
clust.fit(X)


In [ ]:
%%time

labels_050 = cluster_optics_dbscan(
    reachability=clust.reachability_,
    core_distances=clust.core_distances_,
    ordering=clust.ordering_,
    eps=0.5,
)
labels_200 = cluster_optics_dbscan(
    reachability=clust.reachability_,
    core_distances=clust.core_distances_,
    ordering=clust.ordering_,
    eps=2,
)


In [ ]:
import matplotlib.gridspec as gridspec

space = np.arange(len(X))
reachability = clust.reachability_[clust.ordering_]
labels = clust.labels_[clust.ordering_]

plt.figure(figsize=(10, 7))
G = gridspec.GridSpec(2, 3)
ax1 = plt.subplot(G[0, :])
ax2 = plt.subplot(G[1, 0])
ax3 = plt.subplot(G[1, 1])
ax4 = plt.subplot(G[1, 2])

# Reachability plot
colors = ["g.", "r.", "b.", "y.", "c."]
for klass, color in zip(range(0, 5), colors):
    Xk = space[labels == klass]
    Rk = reachability[labels == klass]
    ax1.plot(Xk, Rk, color, alpha=0.3)
ax1.plot(space[labels == -1], reachability[labels == -1], "k.", alpha=0.3)
ax1.plot(space, np.full_like(space, 2.0, dtype=float), "k-", alpha=0.5)
ax1.plot(space, np.full_like(space, 0.5, dtype=float), "k-.", alpha=0.5)
ax1.set_ylabel("Reachability (epsilon distance)")
ax1.set_title("Reachability Plot")


